Deep Learning
=============

Assignment 3
------------

Previously in `2_fullyconnected.ipynb`, you trained a logistic regression and a neural network model.

The goal of this assignment is to explore regularization techniques.

In [2]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle

First reload the data we generated in `1_notmnist.ipynb`.

In [3]:
# data files are in root repository
pickle_file = '../notMNIST.pickle'

with open(pickle_file, 'rb') as f:
    save = pickle.load(f)
    train_dataset = save['train_dataset']
    train_labels = save['train_labels']
    valid_dataset = save['valid_dataset']
    valid_labels = save['valid_labels']
    test_dataset = save['test_dataset']
    test_labels = save['test_labels']
    del save  # hint to help gc free up memory
    print('Training set', train_dataset.shape, train_labels.shape)
    print('Validation set', valid_dataset.shape, valid_labels.shape)
    print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a shape that's more adapted to the models we're going to train:
- data as a flat matrix,
- labels as float 1-hot encodings.

In [4]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
    dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
    # Map 1 to [0.0, 1.0, 0.0 ...], 2 to [0.0, 0.0, 1.0 ...]
    labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
    return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)


In [5]:
def accuracy(predictions, labels):
    return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
              / predictions.shape[0])

---
Problem 1
---------

Introduce and tune L2 regularization for both logistic and neural network models. Remember that L2 amounts to adding a penalty on the norm of the weights to the loss. In TensorFlow, you can compute the L2 loss for a tensor `t` using `nn.l2_loss(t)`. The right amount of regularization should improve your validation / test accuracy.

---

#### First we work on logistic regression

 - We use the minibatch implementation from assignment 2.

In [11]:
# Create TensorFlow graph

batch_size = 128
# regularisation constant
gamma = 0.01

graph1 = tf.Graph()
with graph1.as_default():

    # Input data. For the training data, we use a placeholder that will be fed
    # at run time with a training minibatch.
    tf_train_dataset = tf.placeholder(tf.float32,
                                      shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
  
    # Variables.
    weights = tf.Variable(
         tf.truncated_normal([image_size * image_size, num_labels]))
    biases = tf.Variable(tf.zeros([num_labels]))
  
    # Training computation.
    logits = tf.matmul(tf_train_dataset, weights) + biases
    
    # tf.reduce_mean because we take the average cross entropy over the batch.
    loss = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
    # add regularisation to loss
    # notes: regularise both weights and biases
    loss = loss + gamma * (
    tf.nn.l2_loss(weights) + tf.nn.l2_loss(biases)
    )
        
    # Optimizer.
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(tf.matmul(tf_valid_dataset, weights) + biases)
    test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)

In [7]:
# run tensorFlow graph.

num_steps = 3001

with tf.Session(graph=graph1) as session:
    tf.global_variables_initializer().run()
    print("Initialized")
    
    for step in range(num_steps):
        # Pick an offset within the training data, which has been randomized.
        # Note: we could use better randomization across epochs.
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        # Generate a minibatch.
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        # Prepare a dictionary telling the session where to feed the minibatch.
        # The key of the dictionary is the placeholder node of the graph to be fed,
        # and the value is the numpy array to feed to it.
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        _, l, predictions = session.run(
            [optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 500 == 0):
            print("Minibatch loss at step %d: %f" % (step, l))
            print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
            print("Validation accuracy: %.1f%%" % accuracy(
                valid_prediction.eval(), valid_labels))
    
    print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 50.358704
Minibatch accuracy: 9.4%
Validation accuracy: 9.4%
Minibatch loss at step 500: 0.766424
Minibatch accuracy: 87.5%
Validation accuracy: 81.6%
Minibatch loss at step 1000: 0.800716
Minibatch accuracy: 83.6%
Validation accuracy: 81.3%
Minibatch loss at step 1500: 0.631741
Minibatch accuracy: 85.2%
Validation accuracy: 81.5%
Minibatch loss at step 2000: 0.848516
Minibatch accuracy: 78.1%
Validation accuracy: 79.8%
Minibatch loss at step 2500: 0.717123
Minibatch accuracy: 82.8%
Validation accuracy: 81.7%
Minibatch loss at step 3000: 0.787689
Minibatch accuracy: 78.1%
Validation accuracy: 77.7%
Test accuracy: 83.9%


#### Now let's work on a neural network with a hidden layer

- We use the example from assignment 2:

In [13]:
batch_size = 128
hidden_nodes = 1024
# regularisation constant
gamma = 0.01

graph2 = tf.Graph()
with graph2.as_default():

    # Input data. For the training data, we use a placeholder that will be fed
    # at run time with a training minibatch.
    tf_train_dataset = tf.placeholder(
        tf.float32, shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)

    # Variables - Network Construction!
    # Matrix Dimensions:
    # 1st argument has dimensions coming from previous layer
    # 2nd argument has dimensions going to the next layer == dim(bias)
    # We construct the variables representing the hidden layer:
    weights1 = tf.Variable(
        tf.truncated_normal([image_size * image_size, hidden_nodes]))
    biases1 = tf.Variable(tf.zeros([hidden_nodes]))
    # We construct the variables representing the output layer:
    weights2 = tf.Variable(
        tf.truncated_normal([hidden_nodes, num_labels]))
    biases2 = tf.Variable(tf.zeros([num_labels]))

    # Training computation.
    hidden_layer = tf.matmul(tf_train_dataset, weights1) + biases1
    logits = tf.matmul(hidden_layer, weights2) + biases2
    
    loss = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits(
            labels=tf_train_labels, logits=logits))
    # add regularisation for all weights.
    loss = loss + gamma * (
    tf.nn.l2_loss(weights1) + tf.nn.l2_loss(biases1) + tf.nn.l2_loss(weights2) + tf.nn.l2_loss(biases2))
    
    # Optimizer.
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)

    # Predictions for the training, validation, and test data.
    # Predict for training:
    train_prediction = tf.nn.softmax(logits)
    
    # Create Validation graph
    hidden_layer_val = tf.matmul(tf_valid_dataset, weights1) + biases1
    logits_val = tf.matmul(hidden_layer_val, weights2) + biases2
    # Predict for validation
    valid_prediction = tf.nn.softmax(logits_val)
    
    # Create Test graph
    hidden_layer_test = tf.matmul(tf_test_dataset, weights1) + biases1
    logits_test = tf.matmul(hidden_layer_test, weights2) + biases2
    # Predict for test
    test_prediction = tf.nn.softmax(logits_test)

In [9]:
num_steps = 3001

with tf.Session(graph=graph2) as session:
    tf.global_variables_initializer().run()
    print("Initialized")
    for step in range(num_steps):
        # Pick an offset within the training data, which has been randomized.
        # Note: we could use better randomization across epochs.
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        # Generate a minibatch.
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        # Prepare a dictionary telling the session where to feed the minibatch.
        # The key of the dictionary is the placeholder node of the graph to be fed,
        # and the value is the numpy array to feed to it.
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        _, l, predictions = session.run(
            [optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 500 == 0):
            print("Minibatch loss at step %d: %f" % (step, l))
            print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
            print("Validation accuracy: %.1f%%" % accuracy(
                valid_prediction.eval(), valid_labels))
            
    print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 3640.826172
Minibatch accuracy: 10.2%
Validation accuracy: 35.1%
Minibatch loss at step 500: 20.579571
Minibatch accuracy: 85.2%
Validation accuracy: 79.9%
Minibatch loss at step 1000: 0.989163
Minibatch accuracy: 82.0%
Validation accuracy: 81.3%
Minibatch loss at step 1500: 0.666995
Minibatch accuracy: 85.9%
Validation accuracy: 81.4%
Minibatch loss at step 2000: 0.862258
Minibatch accuracy: 78.9%
Validation accuracy: 79.6%
Minibatch loss at step 2500: 0.751192
Minibatch accuracy: 82.8%
Validation accuracy: 81.7%
Minibatch loss at step 3000: 0.820589
Minibatch accuracy: 80.5%
Validation accuracy: 76.9%
Test accuracy: 82.6%


---
Problem 2
---------
Let's demonstrate an extreme case of overfitting. Restrict your training data to just a few batches. What happens?

---

#### First we work on logistic regression

In [12]:
# run tensorFlow graph for logistic regression.

num_steps = 3001

with tf.Session(graph=graph1) as session:
    tf.global_variables_initializer().run()
    print("Initialized")
    
    for step in range(num_steps):
        # Pick an offset within the training data, which has been randomized.
        # Note: restrict offset to [1, 500]
        offset = np.random.choice(list(range(1, 501)))
        # Generate a minibatch.
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        # Prepare a dictionary telling the session where to feed the minibatch.
        # The key of the dictionary is the placeholder node of the graph to be fed,
        # and the value is the numpy array to feed to it.
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        _, l, predictions = session.run(
            [optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 500 == 0):
            print("Minibatch loss at step %d: %f" % (step, l))
            print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
            print("Validation accuracy: %.1f%%" % accuracy(
                valid_prediction.eval(), valid_labels))
    
    print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 48.841423
Minibatch accuracy: 10.2%
Validation accuracy: 11.8%
Minibatch loss at step 500: 0.478134
Minibatch accuracy: 100.0%
Validation accuracy: 76.7%
Minibatch loss at step 1000: 0.326523
Minibatch accuracy: 100.0%
Validation accuracy: 76.8%
Minibatch loss at step 1500: 0.332783
Minibatch accuracy: 98.4%
Validation accuracy: 76.7%
Minibatch loss at step 2000: 0.326858
Minibatch accuracy: 99.2%
Validation accuracy: 76.3%
Minibatch loss at step 2500: 0.339419
Minibatch accuracy: 99.2%
Validation accuracy: 76.4%
Minibatch loss at step 3000: 0.298383
Minibatch accuracy: 100.0%
Validation accuracy: 76.5%
Test accuracy: 83.4%


#### Now let's work on a neural network with a hidden layer

In [14]:
num_steps = 3001

with tf.Session(graph=graph2) as session:
    tf.global_variables_initializer().run()
    print("Initialized")
    for step in range(num_steps):
        # Pick an offset within the training data, which has been randomized.
        # Note: restrict offset to [1, 500]
        offset = np.random.choice(list(range(1, 501)))
        # Generate a minibatch.
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        # Prepare a dictionary telling the session where to feed the minibatch.
        # The key of the dictionary is the placeholder node of the graph to be fed,
        # and the value is the numpy array to feed to it.
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        _, l, predictions = session.run(
            [optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 500 == 0):
            print("Minibatch loss at step %d: %f" % (step, l))
            print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
            print("Validation accuracy: %.1f%%" % accuracy(
                valid_prediction.eval(), valid_labels))
            
    print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 3725.791992
Minibatch accuracy: 7.8%
Validation accuracy: 43.3%
Minibatch loss at step 500: 20.963247
Minibatch accuracy: 100.0%
Validation accuracy: 74.9%
Minibatch loss at step 1000: 0.449741
Minibatch accuracy: 100.0%
Validation accuracy: 76.4%
Minibatch loss at step 1500: 0.313626
Minibatch accuracy: 100.0%
Validation accuracy: 75.9%
Minibatch loss at step 2000: 0.307216
Minibatch accuracy: 100.0%
Validation accuracy: 75.9%
Minibatch loss at step 2500: 0.268499
Minibatch accuracy: 100.0%
Validation accuracy: 76.5%
Minibatch loss at step 3000: 0.286258
Minibatch accuracy: 100.0%
Validation accuracy: 76.4%
Test accuracy: 83.1%


---
Problem 3
---------
Introduce Dropout on the hidden layer of the neural network. Remember: Dropout should only be introduced during training, not evaluation, otherwise your evaluation results would be stochastic as well. TensorFlow provides `nn.dropout()` for that, but you have to make sure it's only inserted during training.

What happens to our extreme overfitting case?

---

#### Introducing dropout for the hidden layer

In [33]:
batch_size = 128
hidden_nodes = 1024

graph3 = tf.Graph()
with graph3.as_default():

    # Input data. For the training data, we use a placeholder that will be fed
    # at run time with a training minibatch.
    tf_train_dataset = tf.placeholder(
        tf.float32, shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)

    # Variables - Network Construction!
    # Matrix Dimensions:
    # 1st argument has dimensions coming from previous layer
    # 2nd argument has dimensions going to the next layer == dim(bias)
    # We construct the variables representing the hidden layer:
    weights1 = tf.Variable(
        tf.truncated_normal([image_size * image_size, hidden_nodes]))
    biases1 = tf.Variable(tf.zeros([hidden_nodes]))
    # We construct the variables representing the output layer:
    weights2 = tf.Variable(
        tf.truncated_normal([hidden_nodes, num_labels]))
    biases2 = tf.Variable(tf.zeros([num_labels]))

    # Training computation.
    hidden_layer = tf.matmul(tf_train_dataset, weights1) + biases1
    # introduce dropout
    keep_prob = tf.placeholder(tf.float32)
    hidden_layer_d = tf.nn.dropout(hidden_layer, keep_prob)
    
    # logits for prediction
    logits = tf.matmul(hidden_layer, weights2) + biases2
    # logits_d for training with dropout
    logits_d = tf.matmul(hidden_layer_d, weights2) + biases2
    
    loss = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits(
            labels=tf_train_labels, logits=logits_d))
    
    # add regularisation for all weights.
    regconst = tf.placeholder(tf.float32)
    loss = loss + regconst * (
    tf.nn.l2_loss(weights1) + tf.nn.l2_loss(biases1) + tf.nn.l2_loss(weights2) + tf.nn.l2_loss(biases2))
    
    # Optimizer.
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)

    # Predictions for the training, validation, and test data.
    # Predict for training:
    train_prediction = tf.nn.softmax(logits)
    
    # Create Validation graph
    hidden_layer_val = tf.matmul(tf_valid_dataset, weights1) + biases1
    logits_val = tf.matmul(hidden_layer_val, weights2) + biases2
    # Predict for validation
    valid_prediction = tf.nn.softmax(logits_val)
    
    # Create Test graph
    hidden_layer_test = tf.matmul(tf_test_dataset, weights1) + biases1
    logits_test = tf.matmul(hidden_layer_test, weights2) + biases2
    # Predict for test
    test_prediction = tf.nn.softmax(logits_test)

In [34]:
num_steps = 3001
# dropout layer keep probability
keep_probl = 0.5 # cannot have the same name as graph variable!
# regularisation constant
gamma = 0.01

with tf.Session(graph=graph3) as session:
    tf.global_variables_initializer().run()
    print("Initialized")
    for step in range(num_steps):
        # Pick an offset within the training data, which has been randomized.
        # Note: we could use better randomization across epochs.
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        # Generate a minibatch.
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        # Prepare a dictionary telling the session where to feed the minibatch.
        # The key of the dictionary is the placeholder node of the graph to be fed,
        # and the value is the numpy array to feed to it.
        feed_dict = {tf_train_dataset : batch_data,
                     tf_train_labels : batch_labels,
                     regconst : gamma,
                     keep_prob : keep_probl}
        _, l, predictions = session.run(
            [optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 500 == 0):
            print("Minibatch loss at step %d: %f" % (step, l))
            print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
            print("Validation accuracy: %.1f%%" % accuracy(
                valid_prediction.eval(), valid_labels))
            
    print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 3727.914062
Minibatch accuracy: 16.4%
Validation accuracy: 40.5%
Minibatch loss at step 500: 22.028690
Minibatch accuracy: 78.9%
Validation accuracy: 78.9%
Minibatch loss at step 1000: 1.053540
Minibatch accuracy: 82.0%
Validation accuracy: 80.3%
Minibatch loss at step 1500: 0.714607
Minibatch accuracy: 85.2%
Validation accuracy: 81.1%
Minibatch loss at step 2000: 0.941893
Minibatch accuracy: 78.1%
Validation accuracy: 78.2%
Minibatch loss at step 2500: 0.827325
Minibatch accuracy: 82.0%
Validation accuracy: 81.2%
Minibatch loss at step 3000: 0.871661
Minibatch accuracy: 79.7%
Validation accuracy: 76.4%
Test accuracy: 82.1%


#### Restricting training data

In [35]:
num_steps = 3001
# dropout layer keep probability
keep_probl = 0.5 # cannot have the same name as graph variable!
# regularisation constant
gamma = 0.01

with tf.Session(graph=graph3) as session:
    tf.global_variables_initializer().run()
    print("Initialized")
    for step in range(num_steps):
        # Pick an offset within the training data, which has been randomized.
        # Note: restrict offset to [1, 500]
        offset = np.random.choice(list(range(1, 501)))
        # Generate a minibatch.
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        # Prepare a dictionary telling the session where to feed the minibatch.
        # The key of the dictionary is the placeholder node of the graph to be fed,
        # and the value is the numpy array to feed to it.
        feed_dict = {tf_train_dataset : batch_data,
                     tf_train_labels : batch_labels,
                     regconst : gamma,
                     keep_prob : keep_probl}
        _, l, predictions = session.run(
            [optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 500 == 0):
            print("Minibatch loss at step %d: %f" % (step, l))
            print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
            print("Validation accuracy: %.1f%%" % accuracy(
                valid_prediction.eval(), valid_labels))
            
    print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 3761.152588
Minibatch accuracy: 11.7%
Validation accuracy: 38.0%
Minibatch loss at step 500: 22.066456
Minibatch accuracy: 100.0%
Validation accuracy: 74.6%
Minibatch loss at step 1000: 0.446597
Minibatch accuracy: 100.0%
Validation accuracy: 76.3%
Minibatch loss at step 1500: 0.350846
Minibatch accuracy: 100.0%
Validation accuracy: 76.7%
Minibatch loss at step 2000: 0.293637
Minibatch accuracy: 100.0%
Validation accuracy: 76.3%
Minibatch loss at step 2500: 0.365453
Minibatch accuracy: 99.2%
Validation accuracy: 75.7%
Minibatch loss at step 3000: 0.307839
Minibatch accuracy: 100.0%
Validation accuracy: 76.2%
Test accuracy: 83.0%


---
Problem 4
---------

Try to get the best performance you can using a multi-layer model! The best reported test accuracy using a deep network is [97.1%](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html?showComment=1391023266211#c8758720086795711595).

One avenue you can explore is to add multiple layers.

Another one is to use learning rate decay:

    global_step = tf.Variable(0)  # count the number of steps taken.
    learning_rate = tf.train.exponential_decay(0.5, global_step, ...)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
 
 ---


### 1. We start by increasing the training step on the regularised with dropout 1 hidden_layer network.

In [36]:
num_steps = 8001
# dropout layer keep probability
keep_probl = 0.5 # cannot have the same name as graph variable!
# regularisation constant
gamma = 0.01

with tf.Session(graph=graph3) as session:
    tf.global_variables_initializer().run()
    print("Initialized")
    for step in range(num_steps):
        # Pick an offset within the training data, which has been randomized.
        # Note: we could use better randomization across epochs.
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        # Generate a minibatch.
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        # Prepare a dictionary telling the session where to feed the minibatch.
        # The key of the dictionary is the placeholder node of the graph to be fed,
        # and the value is the numpy array to feed to it.
        feed_dict = {tf_train_dataset : batch_data,
                     tf_train_labels : batch_labels,
                     regconst : gamma,
                     keep_prob : keep_probl}
        _, l, predictions = session.run(
            [optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 500 == 0):
            print("Minibatch loss at step %d: %f" % (step, l))
            print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
            print("Validation accuracy: %.1f%%" % accuracy(
                valid_prediction.eval(), valid_labels))
            
    print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 3873.100830
Minibatch accuracy: 6.2%
Validation accuracy: 37.5%
Minibatch loss at step 500: 21.911018
Minibatch accuracy: 83.6%
Validation accuracy: 76.8%
Minibatch loss at step 1000: 1.049506
Minibatch accuracy: 81.2%
Validation accuracy: 80.7%
Minibatch loss at step 1500: 0.705074
Minibatch accuracy: 85.9%
Validation accuracy: 81.2%
Minibatch loss at step 2000: 0.935441
Minibatch accuracy: 78.1%
Validation accuracy: 78.2%
Minibatch loss at step 2500: 0.841109
Minibatch accuracy: 81.2%
Validation accuracy: 80.8%
Minibatch loss at step 3000: 0.883251
Minibatch accuracy: 79.7%
Validation accuracy: 75.9%
Minibatch loss at step 3500: 0.892698
Minibatch accuracy: 81.2%
Validation accuracy: 80.6%
Minibatch loss at step 4000: 1.004611
Minibatch accuracy: 74.2%
Validation accuracy: 80.0%
Minibatch loss at step 4500: 0.991370
Minibatch accuracy: 79.7%
Validation accuracy: 79.4%
Minibatch loss at step 5000: 0.927723
Minibatch accuracy: 75.0%
Validation accu

### 2. Let's increase regularisation.

In [38]:
num_steps = 8001
# dropout layer keep probability
keep_probl = 0.5 # cannot have the same name as graph variable!
# regularisation constant
gamma = 0.03

with tf.Session(graph=graph3) as session:
    tf.global_variables_initializer().run()
    print("Initialized")
    for step in range(num_steps):
        # Pick an offset within the training data, which has been randomized.
        # Note: we could use better randomization across epochs.
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        # Generate a minibatch.
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        # Prepare a dictionary telling the session where to feed the minibatch.
        # The key of the dictionary is the placeholder node of the graph to be fed,
        # and the value is the numpy array to feed to it.
        feed_dict = {tf_train_dataset : batch_data,
                     tf_train_labels : batch_labels,
                     regconst : gamma,
                     keep_prob : keep_probl}
        _, l, predictions = session.run(
            [optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 500 == 0):
            print("Minibatch loss at step %d: %f" % (step, l))
            print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
            print("Validation accuracy: %.1f%%" % accuracy(
                valid_prediction.eval(), valid_labels))
            
    print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 10166.121094
Minibatch accuracy: 7.0%
Validation accuracy: 38.2%
Minibatch loss at step 500: 0.901189
Minibatch accuracy: 85.9%
Validation accuracy: 80.5%
Minibatch loss at step 1000: 1.134885
Minibatch accuracy: 79.7%
Validation accuracy: 79.1%
Minibatch loss at step 1500: 0.873561
Minibatch accuracy: 83.6%
Validation accuracy: 80.6%
Minibatch loss at step 2000: 1.068443
Minibatch accuracy: 78.1%
Validation accuracy: 78.3%
Minibatch loss at step 2500: 1.007971
Minibatch accuracy: 81.2%
Validation accuracy: 80.7%
Minibatch loss at step 3000: 1.036151
Minibatch accuracy: 81.2%
Validation accuracy: 75.4%
Minibatch loss at step 3500: 1.064794
Minibatch accuracy: 80.5%
Validation accuracy: 79.9%
Minibatch loss at step 4000: 1.213073
Minibatch accuracy: 73.4%
Validation accuracy: 78.5%
Minibatch loss at step 4500: 1.136068
Minibatch accuracy: 76.6%
Validation accuracy: 78.0%
Minibatch loss at step 5000: 1.135789
Minibatch accuracy: 75.0%
Validation accu

#### Results:

Increasing regularisation above 0.01 didn't increase performance!